##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Approximate Embeddings Similarity Matching

This tutorial illustrates how to gnerate embeddings from a [TensorFlow Hub](https://www.tensorflow.org/hub) (TF-Hb) module given input data, and build an approximate nearest neighbours (ANN) index using the extracted embeddings. The index can then be used for real-time similarity matching an retreival.

The steps of this tutorial are:
1. Download sample data.
2. Generate embeddings for the data using a TF-Hub module
3. Build an ANN index for the embeddings
4. Use the index for similarity matching

We use [Apache Beam](https://beam.apache.org/documentation/programming-guide/) with [TensorFlow Transform](https://www.tensorflow.org/tfx/tutorials/transform/simple) (TF-Transform) to generate the embeddings from the TF-Hub module. We also use Spotify's [ANNOY](https://github.com/spotify/annoy) library to build the approximate nearest neighbours index. You cam find benchmarking of ANN framework in this [Github repository](https://github.com/erikbern/ann-benchmarks).

## Getting Started

Install the required libraries. Make sure to **restart the runtime** after installtion is completed.

In [0]:
!pip3 install apache_beam[gcp]
!pip3 install tensorflow_transform
!pip3 install annoy

Import the required libraries

In [0]:
import os
import sys
import multiprocessing
import pickle
from datetime import datetime
import numpy as np
import apache_beam as beam
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_hub as hub
import tensorflow_transform.beam as tft_beam
import annoy

In [0]:
print('TF version: {}'.format(tf.__version__))
print('TF-Hub version: {}'.format(hub.__version__))
print('TF-Transform version: {}'.format(tft.__version__))
print('Apache Beam version: {}'.format(beam.__version__))

## 1. Download Sample Data

[A Million News Headlines](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/SYBGZL#) contains data of news headlines published over a period of 15 years. Sourced from the reputable Australian news source Australian Broadcasting Corp. (ABC). This this news dataset as a summarised historical record of noteworthy events in the globe from early-2003 to end-2017 with a more granular focus on Australia. 

This includes the entire corpus of articles published by the ABC website in the given time range. With a volume of 200 articles per day and a good focus on international news, events of significance has been captured here. Digging into the keywords, one can see all the important episodes shaping the last decade and how they evolved over time. Ex: financial crisis, iraq war, multiple US elections, ecological disasters, terrorism, famous people, Australian crimes etc.

**Format**: Tab-separated two-column data: 1) publication date and 2) headline text. We are only interested in the headline text.



In [0]:
!wget https://dataverse.harvard.edu/api/access/datafile/3450625?format=tab&gbrecs=true
!mv 3450625?format=tab raw.tsv
!wc -l raw.tsv
!head raw.tsv

For simplicity, we only keep the headline text and remove the publication date

In [0]:
!rm -r corpus
!mkdir corpus

with open('corpus/text.txt', 'w') as out_file:
  with open('raw.tsv', 'r') as in_file:
    for line in in_file:
      headline = line.split('\t')[1].strip().strip('"')
      out_file.write(headline+"\n")

In [0]:
!tail corpus/text.txt

## 2. Generate Embeddings for the Data.

In this tutorial, we use the [Universal Sentence Encoder](https://tfhub.dev/google/universal-sentence-encoder/2) to generate emebeddings for the headlines data. The sentence embeddings can then be easily used to compute sentence level meaning similarity. We run the embeddings generation process using Apache Beam and TF-Transform.

### Embeddings extraction pipeline

In [0]:
encoder = None

def embed_text(text, module_url):
  '''Generates embedding for the text'''
  import tensorflow_hub as hub
  global encoder
  if not encoder:
    encoder = hub.Module(module_url)
  embedding = encoder(text)
  return embedding


def create_metadata():
  '''Creates metadata for the raw data'''
  from tensorflow_transform.tf_metadata import dataset_metadata
  from tensorflow_transform.tf_metadata import schema_utils
  feature_spec = {'text': tf.FixedLenFeature([], dtype=tf.string)}
  schema = schema_utils.schema_from_feature_spec(feature_spec)
  metadata = dataset_metadata.DatasetMetadata(schema)
  return metadata


def make_preprocess_fn(module_url, random_projection_matrix=None):
  '''Makes a tft preprocess_fn'''

  def _preprocess_fn(input_features):
    '''tft preprocess_fn'''
    text = input_features['text']
    # Generate the embedding for the input text
    embedding = embed_text(text, module_url)

    if random_projection_matrix is not None:
      # Perform random projection for the embedding
      embedding = tf.matmul(
          embedding, tf.cast(random_projection_matrix, embedding.dtype))
    
    output_features = {
        'text': text,
        'embedding': embedding}
    return output_features
  
  return _preprocess_fn


In [0]:
def run_hub2emb(args):
  '''Runs the embedding generation pipeline'''
  source_data_location = args['source_data_location']
  sink_data_location = args['sink_data_location']
  runner = args['runner']
  temporary_dir = args['temporary_location']
  module_url = args['module_url']
  original_dim = args['original_dim']
  projected_dim = args['projected_dim']

  pipeline_options = beam.options.pipeline_options.PipelineOptions(**args)
  raw_metadata = create_metadata()
  converter = tft.coders.CsvCoder(
      column_names=['text'], schema=raw_metadata.schema)
  
  random_projection_matrix = None
  if projected_dim and original_dim != projected_dim:
    # Creating a random projection matrix
    random_projection_matrix = np.random.uniform(
        low=-1, high=1, size=(original_dim, projected_dim))
    print('Storing random projection matrix to disk...')
    with open('random_projection_matrix', 'wb') as handle:
      pickle.dump(random_projection_matrix, 
                  handle, protocol=pickle.HIGHEST_PROTOCOL)

  with beam.Pipeline(runner, options=pipeline_options) as pipeline:
    with tft_beam.Context(temporary_dir):
      # Read the sentences from the input file
      sentences = ( 
          pipeline
          | 'Read sentences from files' >> beam.io.ReadFromText(
              file_pattern=source_data_location)
          | 'Convert to dictionary' >> beam.Map(converter.decode)
      )

      sentences_dataset = (sentences, raw_metadata)
      preprocess_fn = make_preprocess_fn(module_url, random_projection_matrix)
      # Generate the embeddings for the sentence using the TF-Hub module
      embeddings_dataset, _ = (
          sentences_dataset
          | 'Extract embeddings' >> tft_beam.AnalyzeAndTransformDataset(preprocess_fn)
      )

      embeddings, transformed_metadata = embeddings_dataset
      # Write the embeddings to TFRecords files
      embeddings | 'Write embeddings to TFRecords' >> beam.io.tfrecordio.WriteToTFRecord(
          file_path_prefix='{}/emb'.format(sink_data_location),
          file_name_suffix='.tfrecords',
          coder=tft.coders.ExampleProtoCoder(transformed_metadata.schema))
      

### Run pipeline

In [0]:
from apache_beam.runners.portability import fn_api_runner
from apache_beam.portability.api import beam_runner_api_pb2
from apache_beam.portability import python_urns

#runner = 'DirectRunner'
runner = fn_api_runner.FnApiRunner(
          default_environment=beam_runner_api_pb2.Environment(
              urn=python_urns.SUBPROCESS_SDK,
              payload=b'%s -m apache_beam.runners.worker.sdk_worker_main'
                        % sys.executable.encode('ascii')))

job_name = 'hub2emb-{}'.format(datetime.utcnow().strftime('%y%m%d-%H%M%S'))
input_data = 'corpus/*.txt'
output_dir = 'embeds'
temporary_dir = 'tmp'
module_url = 'https://tfhub.dev/google/universal-sentence-encoder/2'

original_dim = 512
projected_dim = 128

args = {
    'job_name': job_name,
    'runner': runner,
    'source_data_location': input_data,
    'sink_data_location': output_dir,
    'temporary_location': temporary_dir,
    'module_url': module_url,
    'original_dim': original_dim,
    'projected_dim': projected_dim,
    'direct_num_workers': multiprocessing.cpu_count()
}

print("Pipeline args are set.")
args

In [0]:
!rm -r {output_dir}
!rm -r {temporary_dir}
!rm random_projection_matrix

print("Running pipeline...")
%time run_hub2emb(args)
print("Pipeline is done.")

In [0]:
!ls {output_dir}

Read some of the generated embeddings...

In [0]:
embed_file = '{}/emb-00000-of-00001.tfrecords'.format(output_dir)
sample = 5
record_iterator =  tf.io.tf_record_iterator(path=embed_file)
for string_record in record_iterator:
  example = tf.train.Example()
  example.ParseFromString(string_record)
  text = example.features.feature['text'].bytes_list.value
  embedding = np.array(example.features.feature['embedding'].float_list.value)
  print("Embedding dimensions: {}".format(embedding.shape[0]))
  print("{}:{}".format(text, embedding[:10]))
  sample-=1
  if sample == 0: break


## 3. Build the ANN Index for the Embeddings

In [0]:
def build_index(embedding_files_pattern, index_filename, vector_length, 
    metric='angular', num_trees=100):
  '''Builds an ANNOY index'''

  annoy_index = annoy.AnnoyIndex(vector_length, metric=metric)
  # Mapping between the item and its identifier in the index
  mapping = {}

  embed_files = tf.gfile.Glob(embedding_files_pattern)
  print('Found {} embedding file(s).'.format(len(embed_files)))

  item_counter = 0
  for f, embed_file in enumerate(embed_files):
    print('Loading embeddings in file {} of {}...'.format(
      f+1, len(embed_files)))
    record_iterator = tf.io.tf_record_iterator(
      path=embed_file)

    for string_record in record_iterator:
      example = tf.train.Example()
      example.ParseFromString(string_record)
      text = example.features.feature['text'].bytes_list.value[0].decode("utf-8")
      mapping[item_counter] = text
      embedding = np.array(
        example.features.feature['embedding'].float_list.value)
      annoy_index.add_item(item_counter, embedding)
      item_counter += 1
      if item_counter % 200000 == 0:
        print('{} items loaded to the index'.format(item_counter))

  print('A total of {} items added to the index'.format(item_counter))

  print('Building the index with {} trees...'.format(num_trees))
  annoy_index.build(n_trees=num_trees)
  print('Index is successfully built.')
  
  print('Saving index to disk...')
  annoy_index.save(index_filename)
  print('Index is saved to disk.')
  print("Index file size: {} GB".format(
    round(os.path.getsize(index_filename) / float(1024 ** 3), 2)))
  annoy_index.unload()

  print('Saving mapping to disk...')
  with open(index_filename + '.mapping', 'wb') as handle:
    pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print('Mapping is saved to disk.')
  print("Mapping file size: {} MB".format(
    round(os.path.getsize(index_filename + '.mapping') / float(1024 ** 2), 2)))

In [0]:
embedding_files = "{}/emb-*.tfrecords".format(output_dir)
embedding_dimension = projected_dim
index_filename = "index"

!rm {index_filename}
!rm {index_filename}.mapping

%time build_index(embedding_files, index_filename, embedding_dimension)

In [0]:
!ls

## 4. Use the Index for Similarity Matching

### Load the index and the mapping files

In [0]:
index = annoy.AnnoyIndex(embedding_dimension)
index.load(index_filename, prefault=True)
print('Annoy index is loaded.')
with open(index_filename + '.mapping', 'rb') as handle:
  mapping = pickle.load(handle)
print('Mapping file is loaded.')


### Similarity matching method

In [0]:
def find_similar_items(embedding, num_matches=5):
  '''Finds similar items to a given embedding in the ANN index'''
  ids = index.get_nns_by_vector(
  embedding, num_matches, search_k=-1, include_distances=False)
  items = [mapping[i] for i in ids]
  return items

### Extract embedding from a given query

In [0]:
# Load the TF-Hub module
print("Loading the TF-Hub module...")
%time embed_fn = hub.load(module_url)
print("TF-hub module is loaded.")

random_projection_matrix = None
if os.path.exists('random_projection_matrix'):
  print("Loading random projection matrix...")
  with open('random_projection_matrix', 'rb') as handle:
    random_projection_matrix = pickle.load(handle)
  print('random projection matrix is loaded.')

def extract_embeddings(query):
  '''Generates the embedding for the query'''
  query_embedding =  embed_fn([query])[0].numpy()
  if random_projection_matrix is not None:
    query_embedding = query_embedding.dot(random_projection_matrix)
  return query_embedding

In [0]:
extract_embeddings("Hello Machine Learning!")[:10]

### Enter a query to find the most similar items

In [0]:
query = "confronting global challenges" #@param {type:"string"}

In [0]:
print("Generating embedding for the query...")
%time query_embedding = extract_embeddings(query)

print("")
print("Finding relevant items in the index...")
%time items = find_similar_items(query_embedding, 10)

print("")
print("Results:")
print("=========")
for item in items:
  print(item)